In [4]:
%pip install -qU datasets==2.12.0 qdrant-client==1.2.0 sentence-transformers==2.2.2 torch==2.0.1

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install -qU datasets qdrant-client sentence-transformers torch


Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install -qU datasets

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
from datasets import load_dataset
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from typing import List

In [8]:
import pandas as pd

# Load your CSV data
df = pd.read_csv('bigBasketProducts.csv')

In [9]:
df = df.fillna('')  # Fill NaN values with an empty string

In [10]:
df['merged_column'] = df.apply(lambda row: ' '.join(row.astype(str)), axis=1)
df

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,merged_column
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.00,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...,1 Garlic Oil - Vegetarian Capsule 500 mg Beaut...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.00,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ...","2 Water Bottle - Orange Kitchen, Garden & Pets..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.00,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m...","3 Brass Angle Deep - Plain, No.2 Cleaning & Ho..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.00,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...,4 Cereal Flip Lid Container/Storage Jar - Asso...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.00,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,5 Creme Soft Soap - For Hands & Body Beauty & ...
...,...,...,...,...,...,...,...,...,...,...,...
27550,27551,"Wottagirl! Perfume Spray - Heaven, Classic",Beauty & Hygiene,Fragrances & Deos,Layerr,199.20,249.0,Perfume,3.9,Layerr brings you Wottagirl Classic fragrant b...,"27551 Wottagirl! Perfume Spray - Heaven, Class..."
27551,27552,Rosemary,Gourmet & World Food,Cooking & Baking Needs,Puramate,67.50,75.0,"Herbs, Seasonings & Rubs",4.0,Puramate rosemary is enough to transform a dis...,27552 Rosemary Gourmet & World Food Cooking & ...
27552,27553,Peri-Peri Sweet Potato Chips,Gourmet & World Food,"Snacks, Dry Fruits, Nuts",FabBox,200.00,200.0,Nachos & Chips,3.8,We have taken the richness of Sweet Potatoes (...,27553 Peri-Peri Sweet Potato Chips Gourmet & W...
27553,27554,Green Tea - Pure Original,Beverages,Tea,Tetley,396.00,495.0,Tea Bags,4.2,"Tetley Green Tea with its refreshing pure, ori...",27554 Green Tea - Pure Original Beverages Tea ...


In [11]:
client = QdrantClient(":memory:")

In [12]:
collection_name = "extractive-question-answering"

collections = client.get_collections()
print(collections)

# only create collection if it doesn't exist
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
collections = client.get_collections()
print(collections)

collections=[]
collections=[CollectionDescription(name='extractive-question-answering')]


In [13]:
# set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# load the retriever model from huggingface model hub
retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [14]:
batch_size = 512  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

for index in tqdm(range(0, len(df), batch_size)):
    i_end = min(index + batch_size, len(df))  # find end of batch
    batch = df.iloc[index:i_end]  # extract batch
    emb = retriever.encode(batch["merged_column"].tolist()).tolist()  # generate embeddings for batch
    meta = batch.to_dict(orient="records")  # get metadata
    ids = list(range(index, i_end))  # create unique IDs

    # upsert to qdrant
    client.upsert(
        collection_name=collection_name,
        points=models.Batch(ids=ids, vectors=emb, payloads=meta),
    )

collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")
assert collection_vector_count == len(df)

  0%|          | 0/54 [00:00<?, ?it/s]

In [ ]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# load the reader model into a question-answering pipeline
reader = pipeline("question-answering", model=model_name, tokenizer=model_name)
print(reader.model, reader)

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

c:\Users\Mohit\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mohit\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [ ]:
def get_relevant_plot(question: str, top_k: int) -> List[str]:
    """
    Get the relevant plot for a given question

    Args:
        question (str): What do we want to know?
        top_k (int): Top K results to return

    Returns:
        context (List[str]):
    """
    try:
        encoded_query = retriever.encode(question).tolist()  # generate embeddings for the question

        result = client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
            [x.payload["product"], x.payload["merged_column"]] for x in result
        ]  # extract title and payload from result
        return context

    except Exception as e:
        print({e})

In [ ]:
def extract_answer(question: str, context: List[str]):
    """
    Extract the answer from the context for a given question

    Args:
        question (str): _description_
        context (list[str]): _description_
    """
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c[1])

        # add the context to answer dict for printing both together, we print only first 500 characters of plot
        answer["title"] = c[0]
        results.append(answer)

    # sort the result based on the score from reader model
    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    for i in range(len(sorted_result)):
        print(f"{i+1}", end=" ")
        print(
            "Answer: ",
            sorted_result[i]["answer"],
            "\n  Title: ",
            sorted_result[i]["title"],
            "\n  score: ",
            sorted_result[i]["score"],
        )


question = "In the movie 3 Idiots, what is the name of the college where the main characters Rancho, Farhan, and Raju study"
context = get_relevant_plot(question, top_k=1)
context

[['Class 11-12 Project Kit',
  '15958 Class 11-12 Project Kit Cleaning & Household Stationery Classmate 297.0 310.0 Notebooks, Files, Folders  Classmates All-Purpose A4 Sheets are perfectly suited for every kind of use such as printing and photocopying. Kids can go on artistical voyages and turn them into their canvas while adults can print important documents and make official notes.\n  Classmates Mechanical pencils are retractable and have a metallic nose for quick and convenient writing and drawing diagrams. It comes equipped with a form-fitted rubber grip, and has 3 spare leads inside the pencil. It also has additional 10 Lead pieces inside the box that comes free with this pack. Classmates Asteroid Geometry Box is a set of essential geometrical instruments used by school children, mathematicians and scientists alike. It comes equipped with all the necessary geometrical instruments such as, self centering compass cum divider, foldable 30 cm scale, special mechanical pencil of 0. 5 

In [ ]:
extract_answer(question, context)

1 Answer:  15958 Class 11-12 
  Title:  Class 11-12 Project Kit 
  score:  9.51765978243202e-05


In [ ]:
question = "Who hates Harry Potter?"
context = get_relevant_plot(question, top_k=1)
extract_answer(question, context)

1 Answer:  Colin 
  Title:  TV & Gadget Cleaner Spray 
  score:  0.6097188591957092


In [ ]:
question = "who killed the spy"
context = get_relevant_plot(question, top_k=3)
extract_answer(question, context)

1 Answer:  Bodyguard 
  Title:  Fruits & Vegetables Disinfectant Tablets 
  score:  0.04258763790130615
2 Answer:  Bodyguard 
  Title:  Multipurpose Alcohol Based Disinfectant Sanitiser Spray - Kills 99.9% Germs 
  score:  0.033500563353300095
3 Answer:  Bodyguard 
  Title:  Multipurpose Alcohol Based Disinfectant Sanitiser Spray - Kills 99.9% Germs 
  score:  0.014654409140348434


In [ ]:
import pickle 

with open("client.pkl" , "wb") as file: 
      pickle.dump(client , file )

NameError: name 'client' is not defined

In [ ]:
with open("retriever.pkl" , "wb") as file: 
      pickle.dump(retriever , file )

In [ ]:
with open("reader.pkl" , "wb") as file: 
      pickle.dump(reader , file )

In [ ]:
import json 

embedding_model_json = {
    "architecture": str(retriever),
    "state_dict": retriever.state_dict()
}